In [12]:
import urllib
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import regex as re
from tqdm.notebook import tqdm
import os

'''
    ['0', 'Отзывы', 'f=25', 'LADA LARGUS'],
    ['1', 'Двигатель', 'f=17', 'LADA LARGUS'],
    ['2', 'Трансмиссия', 'f=18', 'LADA LARGUS'],
    ['3', 'Ходовая', 'f=19', 'LADA LARGUS'],
    ['4', 'Колеса', '', 'LADA LARGUS'],
    ['5', 'Электрооборудование', 'f=16', 'LADA LARGUS'],
    ['6', 'Мультимедиа', '', 'LADA LARGUS'],
    ['7', 'Кузов', 'f=20', 'LADA LARGUS'],
    ['8', 'Жидкости', 'f=21', 'LADA LARGUS'],
    ['9', 'Сервис', 'f=14', 'LADA LARGUS'],
    ['10', 'Эксплуатация', 'f=15', 'LADA LARGUS'],
    ['11', 'Тюнинг', 'f=22', 'LADA LARGUS']
'''

site = 'http://forum.ladalargus.net/'
folder_names = [
    ['0', 'Отзывы', 'f=25', 'LADA LARGUS'],
    ['1', 'Двигатель', 'f=17', 'LADA LARGUS'],
    ['2', 'Трансмиссия', 'f=18', 'LADA LARGUS'],
    ['3', 'Ходовая', 'f=19', 'LADA LARGUS'],
    ['4', 'Колеса', '', 'LADA LARGUS'],
    ['5', 'Электрооборудование', 'f=16', 'LADA LARGUS'],
    ['6', 'Мультимедиа', '', 'LADA LARGUS'],
    ['7', 'Кузов', 'f=20', 'LADA LARGUS'],
    ['8', 'Жидкости', 'f=21', 'LADA LARGUS'],
    ['9', 'Сервис', 'f=14', 'LADA LARGUS'],
    ['10', 'Эксплуатация', 'f=15', 'LADA LARGUS'],
    ['11', 'Тюнинг', 'f=22', 'LADA LARGUS']
]

На странице раздела у нас две задачи:  
1. Найти и сохранить все ссылки на темы
2. Найти ссылку на следующую страницу раздела или убедиться, что это последняя страница раздела.

In [5]:
def collect_themes(site, folder):
    
    site_address = site + 'forumdisplay.php?' + folder
    themes = []
    flag = True

    while flag:
    
        resp = urlopen(site_address) # скачиваем файл
        html = resp.read().decode('utf-8') # считываем содержимое
        soup = BeautifulSoup(html, 'html.parser') # делаем суп
    
        flag = False
        for link in soup.find_all('a'):
            if link.has_attr('href') and link.has_attr('id'):
                if re.search('thread_title_', str(link.get('id'))):
                    s = link.get('href')
                    s = site + 'showthread.php?' + re.search('t=[0-9]+', s)[0]
                    themes.append([link.get_text(), s])
            if not flag and link.has_attr('href') and link.has_attr('rel'):
                if link.get('rel')[0] == 'next':
                    flag = True
                    s = link.get('href')
                    site_address = site + 'forumdisplay.php?' + re.search(folder + '[\S]+', s)[0]
    return themes

На странице темы у нас три задачи:  
1. Понять, что являеся шапкой темы (это сообщение #1), прочитать ее только один раз - НЕ СДЕЛАНО
2. Прочитать и сохранить все сообщения (кроме шапки) на странице темы
3. Найти ссылку на следующую страницу темы или убедиться, что это последняя страница темы.

In [6]:
def collect_messages(site, themes):

    messages = []
    for theme in tqdm(themes):
        
        print(theme[0])
        theme_address = theme[1]
        thread = re.search('t=[0-9]+', theme_address)[0]
        flag = True
        theme_messages = []
        post_data = {}
        
        while flag:
            resp = urlopen(theme_address) # скачиваем файл
            html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
            soup = BeautifulSoup(html, 'html.parser') # делаем суп
            #Собираем некоторые метаданные сообщений на текущей странице
            tables = soup.find_all('table')
            for next_table in tables:
                if next_table.has_attr('id') and re.search('post[0-9]+', next_table['id']):
                    post_column = next_table.find('td')
                    post_date = post_column.get_text().strip()
                    post_data[re.sub('post', '', next_table['id'])] = [post_date]
            #Собираем сами сообщения
            divs = soup.find_all('div')
            for div in divs:
                if div.has_attr('id') and re.search('(postmenu_)([0-9]+)(_menu)', div['id']):
                    continue
                if div.has_attr('id') and re.search('postmenu_', div['id']):
                    author = div.find('a')
                    if author != None:
                        author_id = re.search('u=[0-9]+', author['href'])[0]
                        author_nickname = author.get_text()
                    else:
                        author_id = div.get_text().strip()
                        author_nickname = 'Guest'
                    post_data[re.sub('postmenu_', '', div['id'])].append([author_id, author_nickname])
                if div.has_attr('id') and re.search('post_message_', div['id']):
                    message_id = re.sub('post_message_', '', div['id'])
                    if message_id in post_data:
                        post_date = post_data[message_id][0]
                        author = post_data[message_id][1][1]
                    else:
                        post_date = ''
                        author = ''
                    for item in div.children:
                        if item.name == 'div':
                            item.clear()
                    s = re.sub('[\']+', '`', div.get_text())
                    s = '\'' + re.sub('[\s]+', ' ', s).strip() + '\''
                    if len(s) > 2:
                        theme_messages.append([post_date, author, s])
            # В отдельном цикле ищем ссылку на следующую страницу темы (это 'a' с атрибутом 'rel', равным 'next')
            # Если ссылка найдена, 
            flag = False 
            for link in soup.find_all('a'):
                if not flag and link.has_attr('href') and link.has_attr('rel'):
                    if link.get('rel')[0] == 'next':
                        flag = True
                        s = link.get('href')
                        theme_address = site + 'showthread.php?' + re.search(thread + '[\S]+', s)[0]
        messages.append(['\''+ theme[0] + '\'', theme_messages])
    return messages

In [7]:
def save_messages(messages, folder_name):
    f_name = os.path.join(folder_name[3], folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv')
    with open(f_name, 'w', encoding='utf-8') as ouf:
        ouf.write('Code,Folder,Theme,Date,Time,Author,Message\n')
        for message in messages:
            header = folder_name[:2] + [message[0]]
            for item in message[1]:
                ouf.write(','.join(header + item))
                ouf.write('\n')

In [8]:
def scraper(site, folder_names):
    for folder_name in folder_names:
        if len(folder_name[2]) > 0:
            folder = folder_name[2]
            current_themes = collect_themes(site, folder)
            current_messages = collect_messages(site, current_themes)
            save_messages(current_messages, folder_name)
    print('Information collected')

In [9]:
%load_ext ipytelegram
%reload_ext ipytelegram
import telepot
bot = telepot.Bot('1301715666:AAGBzLlVDZI7KzGZ_DNyukjauVeTt0QpO-A')
response = bot.getUpdates()
%telegram_setup 1301715666:AAGBzLlVDZI7KzGZ_DNyukjauVeTt0QpO-A 1305740495

In [13]:
%%telegram_send Next message
scraper(site, folder_names)

Глушитель
Пожелания по улучшению модели Лада Ларгус
Ваше мнение о Лада Ларгус
Неисправности Ларгуса- писать КРАТКО
Опыт покупки LADA Largus
Неисправность двигателя К4М
Эх, дороги или путешествия на дальние расстояния!
Собираем отзывы о браке передних сидений Ларгуса.
пистолет на АЗС отщелкивает не на полном баке
Вот и заводской дефект
лада ларгус это мул-труженник



Плавают обороты
Диагностика, Настройка (Чип-тюнинг), Ремонт  Лада-Ларгус (СТО Auto063 г. Самара)
проблема с тросиком газа
Замена ремня генератора.
вибрация в салоне
ГБО
Ошибки двигателя К4М
Заглох в пробке
Установка фильтра тонкой очистки топлива
Стрекот со стороны ремня навесного, двигатель 21129
Термостат! Температура открывания?
Будут ли устанавливаться на LADA Largus вазовские моторы? Будет ли дизель?
Двигатель К4М
Замена ИКЗ на КЗ от ВАЗ на К4м
Двигатели Лада Ларгус
Как правильно мыть двигатель Ларгуса?
что за пятно
Срочно,рывки в движении
Пружинные хомуты
Защита двигателя
Как правильно "обкатать" двигатель на Лада Ларгус.
Проблемы с гидрокомпенсаторами клапанов на ДВС.
Регулятор холостого хода (РХХ)
с каким двигателем лучше ?



Вибрация КПП.
Неисправности коробки передач.
ШРУСЫ
Сцепление перегрелось...
Замена шестерёнок пятой передачи
Какой ставить привод?
Пыльники ШРУСА-это болезнь Ларгуса?
гидропривод сцепления у нас регулируется?
масло в КПП
Ахтунг, ахтунг! На легковом Ларгусе КПП от фургона!
Будет ли «автомат» у Лада Ларгус?
Вибрация сцепления



Шины Лада Ларгус
Диски Лада Ларгус
Задние тормоза
Подвеска и рулевое управление Лада Ларгус
Что же свистит?
Колодки тормозные (передние)
Трудно снять передние родные диски
АБС
Съедается резина на задних колесах...
колодки
колесные болты
Всё о зимних шинах. Тесты.
Давление в шинах
Фото Ларгуса на дисках. Только фото.
есть ли в ларгусе ebd?
тормоза
Колпаки на колёсах



Расположение предохранителей
Штатная музыка на Ларгусе
Замена ламп в задних фонарях.
Стартер
Дальний свет фар вместо ДХО
Активация штатных опций электроники Ларгуса
Перестал работать звуковой сигнал/ подрулевой переключатель
Компрессор для шин в прикуриватель
Непонятки по электрооборудованию...
Низкое напряжение (11.6 в) на холодную (после запуска)
Подсветка замка зажигания
Аккумулятор
Замена штатной магнитолы на 2DIN
ПТФ в ДХО за десять минут...
ближний и дальний свет
Доводчик на 2/4 стекла
Кнопки управления электростеклоподъемниками .
Реле с паузой (регулируемой) на задний/передний стеклоочиститель
Выключатель стоп сигналов
Выключатель фонарей заднего хода.
освещение салона.
Штатный бортовой компьютер в комбинации приборов
Hands free
Штатная антенна
"джеки чан" (символ двигателя) загорается иногда.
Перестал работать  подогрев сидения
Периодически отключается разблокировка дверей
Не работает вентилятор отопителя салона
Электрообогрев заднего стекла
освещение бардачка.
Датчик дождя
Пос

FAQ/Оглавление раздела
Антикор
Авто чехлы
Сиденья Ларгуса
2-й ряд сидений на 5-местном люкс. поломка подъёма.
Сверчки
Гамма цветов кузовов Лада Ларгус
Коврик в багажник
Из 5ти местной в 7ми местную
Ночевка в Ларгусе
Кузов Лада Ларгус
Двери багажника
Ларгус и двухметровый холодильник
Атермальные стекла.
Дворник стеклоочистителя.
Сварка и окраска кузовов LADA Largus
Стекло
Где устанавливать домкрат
Система отопления и вентиляции. Кондиционер.
зеркала заднего вида
Бампер передний
Поездка при -20 на 200км
Шумоизоляция
Установка салонного фильтра.
Крепление задних ковриков
Трещины ЛКП над форточкой третьего ряда
Газовый упор капота
Уплотнитель лобового стекла
Потолочная полочка
Передние фартуки (брызговики).
Установка подлокотника в Лада Ларгус
Защитная сетка для радиаторов
Вода в салоне, конденсат кондиционера
Задние брызговики (кто где и какие брал). Сравнение и отзывы.
Фургон LADA Largus. Обшивка багажного отделения (фото)
Регулировка дверей.
Замок задней багажной двери
Особенности конст

Бензин для Ларгуса: 92 или 95?
Расход топлива
Моторное масло для Лада Ларгус
Топливный бак
Масло для ГУР
Тормозная жидкость
Антифриз для Лада Ларгус
Метанол или "официальная" незамерзайка?
Запас хода при горящем датчике топлива.
Какое масло льют на заводе??
Уровень моторного масла.
Почему для Ларгуса рекомендовано 95-98 топливо
FAQ Замена жидкости ГУР
Тестим масла на ойл-клубе



Официальный дилер АВТОКОМПЛЕКС - обслуживание и ремонт клубных LADA Largus
ТО-1 отчеты
Орехово-АвтоЦентр - сервисное обслуживание у официального дилера LADA
Техническое обслуживание автомобилей LADA: ТО-3 в подарок?
Т.О.-2
Вопросы по гарантии/сервису
ТО-3, отчеты
Восстановление Ларгуса после ДТП.
ТО-5
ОСАГО
Информационные письма и предписания
Диалог с ИПГ (книга страждущих :)) )
Поломка "прищепок" ремней безопасности второго ряда сидений - коллективное письмо
Каско
Суровая гарантия
Гарантия и ДОПы
Подрамник.
Когда в свободной продаже будут запчасти на Ларгус?
Лада Имидж
Гарантия "поштучно"
снесла водительское зеркало



Каталог деталей
Лада Ларгус и мороз
Документация в электронном виде
Код заводской магнитолы
Лючок бензобака
Тип и номер краски,аналоги
По передним боковым стёклам.
Неоригинальные запчасти



Автопалатка для LADA Largus: отель на крыше автомобиля?
Обновление+цены апрель+ Бесплатная доставка на сумму от 13000. часть 1
Круиз Контроль
Какие ДОПы выбирать?
Рейлинги, багажник, бокс
Допы для удобства (не для красоты :) )
Колонки
Фаркоп + велокрепление
Установка видеорегистратора
ГБО на Ларгус
Корзина на стандартные рейлинги Ларгуса + бесплатная доставка!
тюнинг
Дефлекторы
Тюнинг стоек дверей.
Автобоксы со скидкой или Бесплатная доставка от Пт Групп?!
омыватель заднего стекла льёт не туда куда надо. решено.
Бесплатная доставка от ПТ Групп
Новогодние скидки на весь АВТОТЮНИНГ!
Чип-тюниг Ларгуса ОТЗЫВЫ.
бортовой компьютер. какой для ларгуса? и попутные вопросы.
Скидка 20% на все! "Неделя черных пятниц"
Скидка на всю продукцию!
Корзина на Ларгус без поперечин!
Готиовимся к зиме. Где найти намордник для Ларгус.
Путешествие одного Ларгуса с Автобоксом в Грузию!
Обновление+цены Декабрь 2017 Дополнительное оборудование "ПТ Групп", Тольятти часть 2
Предстартовый подогрев двигателя от 220в

In [ ]:
import pandas as pd
mess = pd.read_csv('./LADA VESTA/LADA VESTA_10_Эксплуатация.csv', quotechar="'")
mess

In [8]:
folder_names = [['7', 'Кузов', 'f=18', 'LADA VESTA']]
folder = folder_names[0][2]
current_themes = collect_themes(site, folder)

In [ ]:
themes = []
flag = True

while flag:
    
    resp = urlopen(site_address) # скачиваем файл
    html = resp.read().decode('utf-8') # считываем содержимое
    soup = BeautifulSoup(html, 'html.parser') # делаем суп
    
    flag = False
    for link in soup.find_all('a'):
        if link.has_attr('href') and link.has_attr('id'):
            if re.search('thread_title_', str(link.get('id'))):
                s = link.get('href')
                s = site + 'showthread.php?' + re.search('t=[0-9]+', s)[0]
                themes.append([link.get_text(), s])
        if not flag and link.has_attr('href') and link.has_attr('rel'):
            if link.get('rel')[0] == 'next':
                flag = True
                s = link.get('href')
                site_address = site + 'forumdisplay.php?' + re.search(folder + '[\S]+', s)[0]

In [99]:
themes = [['Механика, робот или автомат?',
  'http://www.lada-vesta.net/showthread.php?t=47']]

In [ ]:
messages = []
for theme in tqdm(themes):
    theme_address = theme[1]
    thread = re.search('t=[0-9]+', theme_address)[0]
    flag = True
    theme_messages = []
    post_data = {}
    while flag:
        resp = urlopen(theme_address) # скачиваем файл
        html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
        soup = BeautifulSoup(html, 'html.parser') # делаем суп
        #Собираем некоторые метаданные сообщений на текущей странице
        tables = soup.find_all('table')
        for next_table in tables:
            if next_table.has_attr('id') and re.search('post[0-9]+', next_table['id']):
                post_column = next_table.find('td')
                post_date = post_column.get_text().strip()
                post_data[re.sub('post', '', next_table['id'])] = [post_date]
        #Собираем сами сообщения
        divs = soup.find_all('div')
        for div in divs:
            if div.has_attr('id') and re.search('(postmenu_)([0-9]+)(_menu)', div['id']):
                continue
            if div.has_attr('id') and re.search('postmenu_', div['id']):
                author = div.find('a')
                if author != None:
                    author_id = re.search('u=[0-9]+', author['href'])[0]
                    author_nickname = author.get_text()
                else:
                    author_id = div.get_text().strip()
                    author_nickname = 'Guest'
                post_data[re.sub('postmenu_', '', div['id'])].append([author_id, author_nickname])
                print(re.sub('postmenu_', '', div['id']), post_data[re.sub('postmenu_', '', div['id'])])
            if div.has_attr('id') and re.search('post_message_', div['id']):
                message_id = re.sub('post_message_', '', div['id'])
                if message_id in post_data:
                    post_date = post_data[message_id][0]
                    author = post_data[message_id][1][1]
                else:
                    post_date = ''
                    author = ''
                for item in div.children:
                    if item.name == 'div':
                        item.clear()
                s = re.sub('[\']+', '`', div.get_text())
                s = '\'' + re.sub('[\s]+', ' ', s) + '\''
                if len(s) > 0:
                    theme_messages.append([post_date, author, s])
        # В отдельном цикле ищем ссылку на следующую страницу темы (это 'a' с атрибутом 'rel', равным 'next')
        # Если ссылка найдена, 
        flag = False 
        for link in soup.find_all('a'):
            if not flag and link.has_attr('href') and link.has_attr('rel'):
                if link.get('rel')[0] == 'next':
                    flag = True
                    s = link.get('href')
                    theme_address = site + 'showthread.php?' + re.search(thread + '[\S]+', s)[0]
    messages.append(['\''+ theme[0] + '\'', theme_messages])

In [ ]:
messages

In [85]:
f_name = os.path.join(folder_name[3], folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv')
with open(f_name, 'w', encoding='utf-8') as ouf:
    ouf.write('Code,Folder,Theme,Date,Author,Message\n')
    for message in messages:
        header = folder_name[:2] + [message[0]]
        for item in message[1]:
            ouf.write(','.join(header + item))
            ouf.write('\n')

In [69]:
folder_name = ['0', 'Отзывы', 'f=25', 'LADA VESTA']
f_name = folder_name[3] + '/' + folder_name[3] + '_' + folder_name[0] + '_' + folder_name[1] + '.csv'
f_name

'LADA VESTA/LADA VESTA_0_Отзывы.csv'

In [ ]:
'''messages = []
theme_address = 'http://www.lada-vesta.net/showthread.php?t=9571'
resp = urlopen(theme_address) # скачиваем файл
html = resp.read().decode('utf-8', errors='ignore') # считываем содержимое
soup = BeautifulSoup(html, 'html.parser') # делаем суп
divs = soup.find_all('div')
for div in divs:
    if div.has_attr('id'):
        if re.search('post_message_', div['id']):
            for item in div.children:
                if item.name != 'div':
                    s = str(item).strip()
                    if len(s) > 0:
                        messages.append([s])
print(messages)'''